# One pagers
## Load data

In [ ]:

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
import util
from config import *
import one_pager
from study_gen import study

## Create a one-pager for a reference

In [ ]:

###############
# Parameters
###############
sample = '01_1_S22_reads'
reference = '3042-O-flank_1=hp1-DB'
###############

one_pager.generate_one_pager_reference(
    study=study,
    sample=sample,
    reference=reference,
    output_dir=''
    )